In [1]:
rootF = 'D:/Data/Paper defocus/'

In [2]:
from matplotlib.pyplot import *
from matplotlib import image
from numpy import *
import os.path
import pandas as pd
from skimage import io

from scipy.interpolate import RegularGridInterpolator, interp2d
from scipy.signal import fftconvolve

from shared.CalcXYshift import *

## Select eye simulation

In [3]:
simulation = '20210219 Mouse' 

PupVals = [ '0.80', '1.40', '2.00']
dFvals = ['+40', '+30', '+20', '+10', '+00', '-10' , '-20' , '-30', '-40']
dZvals = ['-0.192344', '-0.146303', '-0.098559', '-0.049559', '0.000000', \
                  '0.049749' , '0.099093', '0.147828', '0.195622']

SourceVals = ['1','2','3','4']

Total_count = len(PupVals) * len(dFvals) * len(SourceVals)

# Other parameters
CrossCorrAlign = 1

# Original images
ImageNbs = [1, 2, 3, 5]
ImageSuffix = '_image.png'

In [4]:
tot_conditions = len(PupVals) * len(dFvals) * len(SourceVals) * len(ImageNbs)
tot_time = (len(PupVals) * len(dFvals) * len(SourceVals) * len(ImageNbs) * 25 * 300) * 2

print ('Number of conditions:', tot_conditions)
print ('Stimulation time:', tot_time, 'ms')
print ('Stimulation time:', tot_time/1000, 'sec')
print ('Stimulation time:', tot_time/1000/60, 'min')

Number of conditions: 432
Stimulation time: 6480000 ms
Stimulation time: 6480.0 sec
Stimulation time: 108.0 min


## Load polychromatic PSFs 

In [9]:
psf_path = rootF + '/Eye model simulations/mouse PSFs/'

PSF = []
PSFname = []

count = 0
for iPup in [1,2]: #range(len(PupVals)):
    for idF in range(len(dFvals)):
        for iSource in range(len(SourceVals)):
            # Load PSF
            norm_poly_psf = load(psf_path + 'pupDiam=' + str(PupVals[iPup]) +\
                                       '_dF=' + str(dFvals[idF]) +  'Source_' + str(SourceVals[iSource]) + '.npy')
            PSF.append(norm_poly_psf)
                    
            # Save name
            PSFname.append('pupDiam=' + str(PupVals[iPup]) +\
                                       '_dF=' + str(dFvals[idF]) +  'Source_' + str(SourceVals[iSource]))

            count += 1
            
PSF = array(PSF)

In [10]:
PSFname

['pupDiam=1.40_dF=+40Source_1',
 'pupDiam=1.40_dF=+40Source_2',
 'pupDiam=1.40_dF=+40Source_3',
 'pupDiam=1.40_dF=+40Source_4',
 'pupDiam=1.40_dF=+30Source_1',
 'pupDiam=1.40_dF=+30Source_2',
 'pupDiam=1.40_dF=+30Source_3',
 'pupDiam=1.40_dF=+30Source_4',
 'pupDiam=1.40_dF=+20Source_1',
 'pupDiam=1.40_dF=+20Source_2',
 'pupDiam=1.40_dF=+20Source_3',
 'pupDiam=1.40_dF=+20Source_4',
 'pupDiam=1.40_dF=+10Source_1',
 'pupDiam=1.40_dF=+10Source_2',
 'pupDiam=1.40_dF=+10Source_3',
 'pupDiam=1.40_dF=+10Source_4',
 'pupDiam=1.40_dF=+00Source_1',
 'pupDiam=1.40_dF=+00Source_2',
 'pupDiam=1.40_dF=+00Source_3',
 'pupDiam=1.40_dF=+00Source_4',
 'pupDiam=1.40_dF=-10Source_1',
 'pupDiam=1.40_dF=-10Source_2',
 'pupDiam=1.40_dF=-10Source_3',
 'pupDiam=1.40_dF=-10Source_4',
 'pupDiam=1.40_dF=-20Source_1',
 'pupDiam=1.40_dF=-20Source_2',
 'pupDiam=1.40_dF=-20Source_3',
 'pupDiam=1.40_dF=-20Source_4',
 'pupDiam=1.40_dF=-30Source_1',
 'pupDiam=1.40_dF=-30Source_2',
 'pupDiam=1.40_dF=-30Source_3',
 'pupDia

## Convolve images with PSF and re-align them

In [11]:
images_path = rootF + '/Eye model simulations/original images/image_'

In [12]:
# PSF orientation
rotation = 0

# Inverting image's polarity
polarity_invertion = False

# Save realignment distances
shifts = zeros((len(PSF), 2))

D0=10
yshift_force=-10          #-10 to not use
border_thickness = 10

for iIm in ImageNbs[1:]:

    ImageFile = images_path + str(iIm) + ImageSuffix
    
    if polarity_invertion:
        oPic = imread(ImageFile)
        Pic = abs(oPic - 1)
        # imshow(Pic, cmap='Greys_r', vmin=0, vmax=1)
    else:
        Pic = imread(ImageFile)

    # UpSample image so that 1 pixel is 1 um
    f = interp2d(linspace(0, 864, 864), linspace(0, 864, 864), Pic, kind='cubic')
    iPic = f(arange(0, len(Pic), 1/3.5), arange(0, len(Pic), 1/3.5))

    for count in range(len(PSF)):
        print (count)
        print (PSFname[count])

        OptFilt = PSF[count,:,:]
        # Rotate PSF
        OptFilt = rot90(OptFilt, rotation) # 180°
        OptFilt = OptFilt / sum(OptFilt) #norm to 1

        cPic = fftconvolve(iPic,OptFilt,mode='same')
        
        # Downsample image to original size 864 x 864
        ff = interp2d(linspace(0, len(iPic), len(iPic)), linspace(0, len(iPic), len(iPic)), cPic, kind='cubic')
        convPic = ff(arange(0, len(iPic), 3.5), arange(0, len(iPic), 3.5))
        
#         # Save blurred image        
#         image.imsave(rootF + 'Documents/Myopia/Data/MEA data/inverted blurred images/Unshifted conv images/' \
#                      + 'image_' + str(iIm) + PSFname[count] + '.png', convPic)
        
        # Realign images 
        if CrossCorrAlign > 0:
            print ('Aligning image')
            print (PSFname[count][8:12], PSFname[count][-1], int(PSFname[count][16:19]))
                        
            # fixes exceptions
            if (PSFname[count][8:12] == '1.40') & (PSFname[count][-1] == '4') & \
                        (int(PSFname[count][16:19]) <= -35):
                print ('exception')
                Dval=22
            elif (PSFname[count][8:12] == '1.40') & (PSFname[count][-1] == '1') & \
                        (int(PSFname[count][16:19]) <= -35):
                print ('exception')
                Dval=22
            elif (PSFname[count][8:12] == '2.00') & (PSFname[count][-1] == '1') & \
                        (int(PSFname[count][16:19])) <= -30:
                print ('exception')
                Dval=22
            elif (PSFname[count][8:12] == '1.40') & (PSFname[count][-1] == '2') & \
                            (int(PSFname[count][16:19]) > -40):
                print ('exception')
                Dval=20
            elif (PSFname[count][8:12] == '1.40') & (PSFname[count][-1] == '3') & \
                            (int(PSFname[count][16:19]) <- 25):
                print ('exception')
                Dval = 30
            elif (PSFname[count][8:12] == '2.00') & (PSFname[count][-1] == '3') & \
                            (int(PSFname[count][16:19]) == -40):
                Dval = 20
                print ('exception')
            elif (PSFname[count][8:12] == '2.00') & (PSFname[count][-1] == '3') & \
                            (int(PSFname[count][16:19]) <= -25):
                Dval = 30
                print ('exception')
            else:
                Dval = D0
                print ('no exception')
                
            #print ('Dval = ', Dval)

            xshift, yshift = CalcXYshift(Pic,convPic,Dval,border_thickness)
            #print ('shift:', xshift)
            
            shifts[count,:] = xshift, yshift

            if yshift_force>-10:
                yshift=yshift_force

            if xshift>0:
                convPic[:, :-xshift] = convPic[:, xshift:]
                convPic[:, -xshift:] = 0
            else:
                convPic[:, -xshift:] = convPic[:, :xshift]
                convPic[:, :-xshift] = 0
        
        # cut borders to perceptually unbias perception of shift at evaluation stage
        convPic[0:border_thickness,:] = 0
        convPic[-border_thickness:,:] = 0
        convPic[:,-border_thickness:] = 0
        convPic[:,0:border_thickness] = 0
        
        # Save blurred image        
        rescaled_convPic = convPic * 255

        io.imsave(rootF + '/Eye model simulations/convolved images/image_' \
                   + str(iIm) + PSFname[count] + '.png', rescaled_convPic.astype(np.uint8))

print ('Done!')

C:\Users\Sarah\AppData\Local\Temp\ipykernel_11644\1113374266.py:26: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  f = interp2d(linspace(0, 864, 864), linspace(0, 864, 864), Pic, kind='cubic')
C:\Users\Sarah\AppData\Local\Temp\ipykernel_11644\1113374266.py:27: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, 

0
pupDiam=1.40_dF=+40Source_1


C:\Users\Sarah\AppData\Local\Temp\ipykernel_11644\1113374266.py:41: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  ff = interp2d(linspace(0, len(iPic), len(iPic)), linspace(0, len(iPic), len(iPic)), cPic, kind='cubic')
C:\Users\Sarah\AppData\Local\Temp\ipykernel_11644\1113374266.py:42: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariate

Aligning image
1.40 1 40
no exception
3181.0992309482485 (422, 422) 422 422
1
pupDiam=1.40_dF=+40Source_2
Aligning image
1.40 2 40
exception
1616.173400462836 (422, 421) 422 421
2
pupDiam=1.40_dF=+40Source_3
Aligning image
1.40 3 40
no exception
3013.0376469761063 (422, 417) 422 417
3
pupDiam=1.40_dF=+40Source_4
Aligning image
1.40 4 40
no exception
2888.4504897487573 (422, 417) 422 417
4
pupDiam=1.40_dF=+30Source_1
Aligning image
1.40 1 30
no exception
5169.743226794861 (423, 423) 423 423
5
pupDiam=1.40_dF=+30Source_2
Aligning image
1.40 2 30
exception
2437.9454510673236 (423, 420) 423 420
6
pupDiam=1.40_dF=+30Source_3
Aligning image
1.40 3 30
no exception
3621.7100111152845 (423, 417) 423 417
7
pupDiam=1.40_dF=+30Source_4
Aligning image
1.40 4 30
no exception
3301.4041847282083 (422, 418) 422 418
8
pupDiam=1.40_dF=+20Source_1
Aligning image
1.40 1 20
no exception
7152.227187633115 (423, 423) 423 423
9
pupDiam=1.40_dF=+20Source_2
Aligning image
1.40 2 20
exception
3648.9067927959622 (

866.4629274640313 (423, 420) 423 420
6
pupDiam=1.40_dF=+30Source_3
Aligning image
1.40 3 30
no exception
1576.4960411109055 (423, 418) 423 418
7
pupDiam=1.40_dF=+30Source_4
Aligning image
1.40 4 30
no exception
1426.452506203288 (423, 418) 423 418
8
pupDiam=1.40_dF=+20Source_1
Aligning image
1.40 1 20
no exception
2747.7867045011703 (423, 423) 423 423
9
pupDiam=1.40_dF=+20Source_2
Aligning image
1.40 2 20
exception
1186.6531342276846 (423, 420) 423 420
10
pupDiam=1.40_dF=+20Source_3
Aligning image
1.40 3 20
no exception
1750.8707651063528 (423, 419) 423 419
11
pupDiam=1.40_dF=+20Source_4
Aligning image
1.40 4 20
no exception
1536.869678976997 (423, 420) 423 420
12
pupDiam=1.40_dF=+10Source_1
Aligning image
1.40 1 10
no exception
2784.5708720784705 (423, 423) 423 423
13
pupDiam=1.40_dF=+10Source_2
Aligning image
1.40 2 10
exception
1430.3938846196188 (423, 421) 423 421
14
pupDiam=1.40_dF=+10Source_3
Aligning image
1.40 3 10
no exception
1936.1536815938198 (423, 421) 423 421
15
pupDiam=1

11
pupDiam=1.40_dF=+20Source_4
Aligning image
1.40 4 20
no exception
9460.17247859109 (423, 420) 423 420
12
pupDiam=1.40_dF=+10Source_1
Aligning image
1.40 1 10
no exception
16069.76834145248 (423, 423) 423 423
13
pupDiam=1.40_dF=+10Source_2
Aligning image
1.40 2 10
exception
7618.676808970921 (423, 421) 423 421
14
pupDiam=1.40_dF=+10Source_3
Aligning image
1.40 3 10
no exception
11494.448122156062 (423, 420) 423 420
15
pupDiam=1.40_dF=+10Source_4
Aligning image
1.40 4 10
no exception
9839.425938648254 (423, 421) 423 421
16
pupDiam=1.40_dF=+00Source_1
Aligning image
1.40 1 0
no exception
11815.994480970585 (423, 423) 423 423
17
pupDiam=1.40_dF=+00Source_2
Aligning image
1.40 2 0
exception
6074.68573921811 (423, 422) 423 422
18
pupDiam=1.40_dF=+00Source_3
Aligning image
1.40 3 0
no exception
10678.36089081741 (423, 422) 423 422
19
pupDiam=1.40_dF=+00Source_4
Aligning image
1.40 4 0
no exception
9672.595388067692 (423, 423) 423 423
20
pupDiam=1.40_dF=-10Source_1
Aligning image
1.40 1 -10